# Python’s SQLite API

Python has such an extensive library of modules and methods that allow you to manage data. SQL is one of the most efficient languages when it comes to working with relational databases. What if there were some way to combine the two to increase your functionality with relational databases? The good news is that we can thanks to **Python’s Database-API (DB-API)**. With **DB-API 2.0**, we can connect Python to RDBMS like `PostgreSQL(psycopg2)`, `MySQL(mysqlclient)`, `Oracle(pyodbc`), and` SQLite`. In this article we will explore the sqlite3 module, which allows us to create, read, update, and delete the data in our SQLite relational databases, but within our Python script, how cool is that? Let’s get started by viewing some of the API’s that are available to us within the module.

## Connecting to SQLite in Python

In [62]:
import sqlite3
import pandas as pd

In [52]:
# Create connection to database
conn = sqlite3.connect("first.db")

In [53]:
# Instantiate a cursor object
cursor = conn.cursor()

## Executing SQL Statements in Python

In [54]:
# Create students table
cursor.execute('''CREATE TABLE students (
                    id INTEGER PRIMARY KEY,
                    name TEXT NOT NULL,
                    email TEXT NOT NULL UNIQUE,
                    major_code INTEGER,
                    grad_date datetime,
                    grade REAL NOT NULL)''')

OperationalError: table students already exists

In [ ]:
# Add a row of data to students table
cursor.execute('''INSERT INTO students VALUES (101, 'Alex', 'alex@codeu.com', 32, '2022-05-16', 'Pass')''')

To insert multiple values at once we can use the `executemany()` method, a variation of the execute method which allows us to execute multiple commands in a single API call. Let’s create a list of student data that follows the students table schema so that the data can be added swiftly:



In [55]:
students = [(102, 'Joe', 'joseph@codeu.com', 32, '2022-05-16', 'Pass'),
            (103, 'Stac# Create a list of tuples of the major codes
major_codes = cursor.execute("SELECT major_code FROM students;").fetchall()
 
# Obtain the average of the tuple list by using for loops
sum = 0
for num in major_codes: 
    for i in num: 
        sum = sum + i 
average = sum / len(major_codes)
 
# Show average
print(average)y', 'stacy@codeu.com', 10, '2022-05-16', 'Pass'),
            (104, 'Angela', 'angela@codeu.com', 21, '2022-12-20', 'Pass'),
            (105, 'Mark', 'mark@codeu.com', 21, '2022-12-20', 'Fail'),
            (106, 'Nathan', 'nathaniel@codeu.com', 21, '2022-12-20', 'Pass')
            ]
 
# Insert values into the students table
cursor.executemany('''INSERT INTO students VALUES (?,?,?,?,?,?)''', students)

# Commit changes to database
conn.commit()

SyntaxError: EOL while scanning string literal (<ipython-input-55-823d6583621f>, line 2)

We use 6 question marks as placeholders to represent each of the 6 fields in the database that we will insert values into. Having made changes to our database, we need to ensure that the changes will be visible to others who may be working with our database. We use the `commit()` method on our connection to make sure these changes are committed.

## Reading our SQL data with Python

To read the data within our database, we can use multiple methods. The most simple is to use a for loop that iterates through our database and calls some SQL statement:

In [56]:
# data = cursor.execute("SELECT * FROM students")
for row in cursor.execute("SELECT * FROM students"):
    print(row)

(102, 'Joe', 'joseph@codeu.com', 32, '2022-05-16', 'Pass')
(103, 'Stacy', 'stacy@codeu.com', 10, '2022-05-16', 'Pass')
(104, 'Angela', 'angela@codeu.com', 21, '2022-12-20', 'Pass')
(105, 'Mark', 'mark@codeu.com', 21, '2022-12-20', 'Fail')
(106, 'Nathan', 'nathaniel@codeu.com', 21, '2022-12-20', 'Pass')


There are a number of sqlite3 methods that will retrieve data, these being

* `fetchone()`
* `fetchmany()`
* `fetchall()`

In [57]:
#When we simply want to return the first row that fulfills a query, we can use fetchone()
cursor.execute("SELECT * FROM students").fetchone()

(102, 'Joe', 'joseph@codeu.com', 32, '2022-05-16', 'Pass')

To return a specific number of rows from our database that correspond to a particular query, we can call **`fetchmany()`** on our statement, and set the number of rows as a parameter. Like `fetchone()`, this call will return the first rows that match our query.

In [58]:
# Return first three rows in students
cursor.execute("SELECT * FROM students").fetchmany(3)

[(102, 'Joe', 'joseph@codeu.com', 32, '2022-05-16', 'Pass'),
 (103, 'Stacy', 'stacy@codeu.com', 10, '2022-05-16', 'Pass'),
 (104, 'Angela', 'angela@codeu.com', 21, '2022-12-20', 'Pass')]

If we want to return all rows associated with a certain SQL statement, we can call the `fetchall()` method:


In [59]:
# Return all rows in students
cursor.execute("SELECT * FROM students").fetchall()

[(102, 'Joe', 'joseph@codeu.com', 32, '2022-05-16', 'Pass'),
 (103, 'Stacy', 'stacy@codeu.com', 10, '2022-05-16', 'Pass'),
 (104, 'Angela', 'angela@codeu.com', 21, '2022-12-20', 'Pass'),
 (105, 'Mark', 'mark@codeu.com', 21, '2022-12-20', 'Fail'),
 (106, 'Nathan', 'nathaniel@codeu.com', 21, '2022-12-20', 'Pass')]

*Notice that using for loops and the `fetchone()` method return tuples, while `fetchmany()` and `fetchall()` return lists of tuples.*

All of your SQLite syntax will work in Python. You can use clauses like `WHERE`, `COUNT`, etc.

In [60]:
# Return the number of rows with a passing grade
cursor.execute("""SELECT COUNT(*) FROM students WHERE Grade = 'Pass';""").fetchone()

(4,)

Let’s say that we would like to find the average of the major codes field. We can use Python methods `sum()` and `len()` on our result set to obtain the mean value of the field.

In [61]:
# Create a list of tuples of the major codes
major_codes = cursor.execute("SELECT major_code FROM students;").fetchall()
 
# Obtain the average of the tuple list by using for loops
sum = 0
for num in major_codes: 
    for i in num: 
        sum = sum + i 
average = sum / len(major_codes)
 
# Show average
print(average)

21.0


In [66]:
# Create a new dataframe from the result set
df = pd.read_sql_query('''SELECT * from students WHERE major_code = 21;''', conn)
 
# Show new dataframe
print(df)

    id    name                email  major_code   grad_date grade
0  104  Angela     angela@codeu.com          21  2022-12-20  Pass
1  105    Mark       mark@codeu.com          21  2022-12-20  Fail
2  106  Nathan  nathaniel@codeu.com          21  2022-12-20  Pass


In [67]:
# Return the average of major code
df['major_code'].mean()

21.0